In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("tmp.csv")
df = df.drop(["mycut"],axis=1)
df

,微博中文内容,情感倾向
0,写在年末冬初孩子流感的第五天，我们仍然没有忘记热情拥抱这2020年的第一天。带着一丝迷信，早...,0
1,开年大模型…累到以为自己发烧了腰疼膝盖疼腿疼胳膊疼脖子疼#Luna的Krystallife#?,-1
2,邱晨这就是我爹，爹，发烧快好，毕竟美好的假期拿来养病不太好，假期还是要好好享受快乐，爹，新...,1
3,新年的第一天感冒又发烧的也太衰了但是我要想着明天一定会好的?,1
4,问：我们意念里有坏的想法了，天神就会给记下来，那如果有好的想法也会被记下来吗？答：那当然了。...,1
...,...,...
99995,#抗击新型肺炎第一线#【@中国计量大学研制新冠病毒蛋白标准样品】记者从中国计量大学获悉，新型...,0
99996,1、类RaTG13病毒（一种从云南蝙蝠身上分离出来的冠状病毒）可能是2019-nCoV的源头...,0
99997,#微博辟谣#没有证据表明，吃大蒜、漱口水、涂抹芝麻油、生理盐水洗鼻子等手段可以防止感染新型冠...,0
99998,【新冠疫情最受关注的十一篇英文核心期刊论文全解析】本文整理了关于新型冠状病毒最受关注的十一篇...,1


In [2]:
df1 = df[df.情感倾向.isin(['1'])] #筛选情感倾向为1的内容
df1 = df1.drop(["情感倾向"],axis=1)
df1.to_csv(path_or_buf="pos.csv",index=False)

In [3]:
df2 = df[df.情感倾向.isin(['-1'])] #筛选情感倾向为-1的内容
df2 = df2.drop(["情感倾向"],axis=1)
df2.to_csv(path_or_buf="neg.csv",index=False)

In [1]:
from nltk.probability import  FreqDist,ConditionalFreqDist
from nltk.metrics import  BigramAssocMeasures

def bigram_words(words, score_fn=BigramAssocMeasures.chi_sq, n=1000):

    tuple_words = []
    for i in words:
        temp = (i,)
        tuple_words.append(temp)

    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n) # 使用了卡方统计的方法，选择排名前1000的双

    return bag_of_words(tuple_words + bigrams)  # 所有词和（信息量大的）双词搭配一起作为特征

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [2]:
from nltk.collocations import BigramCollocationFinder
import jieba

def read_file(filename):
  
    stop = [line.strip() for line in  open('stopwordsHIT.txt','r',encoding='utf-8').readlines()]#停用词
    f = open(filename,'r',encoding='utf-8')
    line = f.readline()
    str = []
    while line:
        s = line.split('\t')
        fenci = jieba.cut(s[0],cut_all=False)#False默认值：精准模式
        str.append(list(set(fenci)-set(stop)))
        line = f.readline()
    return str

def create_word_bigram_scores():
    posWords = []
    negWords = []
    for items in read_file('pos.txt'):#把集合的集合变成集合
        for item in items:
            posWords.append(item)
    for items in read_file('neg.txt'):
        for item in items:
            negWords.append(item)

    bigram_finder = BigramCollocationFinder.from_words(posWords)
    posBigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 5000)
    bigram_finder = BigramCollocationFinder.from_words(negWords)
    negBigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 5000)

    pos = posWords + posBigrams  # 词和双词搭配
    neg = negWords + negBigrams

    word_fd = FreqDist()
    cond_word_fd = ConditionalFreqDist()
    for word in pos:
        word_fd[word] += 1
        cond_word_fd["pos"][word] += 1
    for word in neg:
        word_fd[word] += 1
        cond_word_fd["neg"][word] += 1

    pos_word_count = cond_word_fd['pos'].N()
    neg_word_count = cond_word_fd['neg'].N()
    total_word_count = pos_word_count + neg_word_count

    word_scores = {}
    for word, freq in word_fd.items():
        pos_score = BigramAssocMeasures.chi_sq(cond_word_fd['pos'][word], (freq, pos_word_count), total_word_count)  # 计算积极词的卡方统计量，这里也可以计算互信息等其它统计量
        neg_score = BigramAssocMeasures.chi_sq(cond_word_fd['neg'][word], (freq, neg_word_count), total_word_count)
        word_scores[word] = pos_score + neg_score

    return word_scores

def find_best_words(word_scores, number):
    best_vals = sorted(word_scores.items(), key=lambda w_s: w_s[1], reverse=True)[:number]  # 把词按信息量倒序排序。number是特征的维度，是可以不断调整直至最优的
    best_words = set([w for w, s in best_vals])
    return best_words


# 2.3 把选出的这些词作为特征（这就是选择了信息量丰富的特征）
def best_word_features(words):
    # load_data()
    # word_scores = create_word_bigram_scores()
    global best_words
    # best_words = find_best_words(word_scores, 7500)
    return dict([(word, True) for word in words if word in best_words])



# 3.3 赋予类标签
# 3.3.1 积极
def pos_features(feature_extraction_method):
    posFeatures = []
    for i in read_file('pos.txt'):
        posWords = [feature_extraction_method(i), 'pos']  # 为积极文本赋予"pos"
        posFeatures.append(posWords)
    return posFeatures


# 3.3.2 消极
def neg_features(feature_extraction_method):

    negFeatures = []
    for j in read_file('neg.txt'):
        negWords = [feature_extraction_method(j), 'neg']  # 为消极文本赋予"neg"
        negFeatures.append(negWords)
    return negFeatures


train = []  # 训练集
devtest = []  # 开发测试集
test = []  # 测试集
dev = []  #
tag_dev = []



# 3.4 把特征化之后的数据数据分割为开发集和测试集
def cut_data(posFeatures, negFeatures):
    global train, devtest, test
    # train = posFeatures[300:] + negFeatures[300:]
    # devtest = posFeatures[300:500] + negFeatures[300:500]
    # test = posFeatures[:500] + negFeatures[:500]
    train = posFeatures[1500:] + negFeatures[1500:]
    devtest = posFeatures[:500] + negFeatures[:500]


# 4.1 开发测试集分割人工标注的标签和数据
def cut_devtest():
    global dev, tag_dev
    dev, tag_dev = zip(*devtest)
    
word_scores = create_word_bigram_scores()
best_words = find_best_words(word_scores, 1500)  # 选择信息量最丰富的1500个的特征
posFeatures = pos_features(best_word_features)
negFeatures = neg_features(best_word_features)

cut_data(posFeatures, negFeatures)
cut_devtest()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.628 seconds.
Prefix dict has been built successfully.


In [14]:
import nltk
import os
import sklearn
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [4]:


def score(classifier):
    classifier = nltk.SklearnClassifier(classifier)  # 在nltk 中使用scikit-learn的接口
    classifier.train(train)  #训练分类器

    pred = classifier.classify_many(dev)  # 对开发测试集的数据进行分类，给出预测的标签
    return accuracy_score(tag_dev, pred)  # 对比分类预测结果和人工标注的正确结果，给出分类器准确度


def try_diffirent_classifiers():

    results = list()
    results.append(score(BernoulliNB()))
    results.append(score(MultinomialNB()))
    results.append(score(LogisticRegression()))
    results.append(score(SVC()))
    results.append(score(LinearSVC()))
    results.append(score(NuSVC()))

    return results

results = try_diffirent_classifiers()
results



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for un

[0.861, 0.861, 0.904, 0.822, 0.9, 0.877]

In [15]:
from sklearn.externals import joblib

MultinomialNB_classifier = SklearnClassifier(MultinomialNB())
MultinomialNB_classifier.train(train)
joblib.dump(MultinomialNB_classifier, 'MultinomialNB.pkl')

NameError: name 'train' is not defined

In [18]:
def transfer_text_to_moto():

    def extract_features():
        feat = []
        for i in read_file('微博评论.txt'):
            feat.append(best_word_features(i))
        return feat

    moto_features = extract_features()  # 把文本转化为特征表示的形式
    return moto_features
def application(moto_features):
    clf = joblib.load('MultinomialNB.pkl')  # 载入分类器

    pred = clf.prob_classify_many(moto_features)  # 该方法是计算分类概率值的
    p_file = open('test_result.txt', 'w')  # 把结果写入文档
    for i in pred:
        p_file.write(str(i.prob('pos')) + ' ' + str(i.prob('neg')) + '\n')
    p_file.close()

moto_features = transfer_text_to_moto()
application(moto_features)


In [20]:
motion = pd.read_csv("test_result.csv",delimiter=',',error_bad_lines=False)
motion

,积极数据,消极数据
0,0.740446,0.259554
1,0.122701,0.877299
2,0.602974,0.397026
3,0.971032,0.028968
4,0.612186,0.387814
...,...,...
22471,0.084605,0.915395
22472,0.022441,0.977559
22473,0.000020,0.999980
22474,0.944181,0.055819


In [12]:
info = pd.read_csv("数据.csv")
info

,人名,点赞数,时间,评论,地区
0,是吴珉珉呀,123,2020-12-31 07:57,辽宁疫情快结束吧！2021好运来！,北京 海淀区
1,尼奇窝窝,41,2020-12-31 07:58,做好个人防护，中国这种做法不可能大面积传染开,上海 宝山区
2,尼奇窝窝,11,2020-12-3 09:46,保护好自己，不给国家添乱,上海 宝山区
3,尼奇窝窝,9,2020-11-30 09:01,医护人员辛苦了,上海 宝山区
4,尼奇窝窝,3,2020-11-22 10:48,太不容易了,上海 宝山区
...,...,...,...,...,...
22471,薏苡啊,0,2020-3-8 17:01,我是黄冈的 能把快递通了吗？我姨妈巾都没得用了,其他
22472,温良初久,0,2020-3-8 17:00,只希望不要因为海外输入，病毒再次卷土重来，不知道山东靠海地区，能不能做...,山东
22473,小森宝的小佳宝DZ,0,2020-3-8 17:00,得 好不容易零增长了 现在又加上个本土 服了服了 那些看着中国疫...,山东 济南
22474,李土條,0,2020-3-8 16:59,太好了！,其他


In [26]:
data = info.join(motion)
data.

,人名,点赞数,时间,评论,地区,积极数据,消极数据
0,是吴珉珉呀,123,2020-12-31 07:57,辽宁疫情快结束吧！2021好运来！,北京 海淀区,0.740446,0.259554
1,尼奇窝窝,41,2020-12-31 07:58,做好个人防护，中国这种做法不可能大面积传染开,上海 宝山区,0.122701,0.877299
2,尼奇窝窝,11,2020-12-3 09:46,保护好自己，不给国家添乱,上海 宝山区,0.602974,0.397026
3,尼奇窝窝,9,2020-11-30 09:01,医护人员辛苦了,上海 宝山区,0.971032,0.028968
4,尼奇窝窝,3,2020-11-22 10:48,太不容易了,上海 宝山区,0.612186,0.387814
...,...,...,...,...,...,...,...
22471,薏苡啊,0,2020-3-8 17:01,我是黄冈的 能把快递通了吗？我姨妈巾都没得用了,其他,0.084605,0.915395
22472,温良初久,0,2020-3-8 17:00,只希望不要因为海外输入，病毒再次卷土重来，不知道山东靠海地区，能不能做...,山东,0.022441,0.977559
22473,小森宝的小佳宝DZ,0,2020-3-8 17:00,得 好不容易零增长了 现在又加上个本土 服了服了 那些看着中国疫...,山东 济南,0.000020,0.999980
22474,李土條,0,2020-3-8 16:59,太好了！,其他,0.944181,0.055819


,人名,点赞数,时间,评论,地区,积极数据,消极数据
0,是吴珉珉呀,123,2020-12-31 07:57,辽宁疫情快结束吧！2021好运来！,北京 海淀区,0.740446,0.259554
1,尼奇窝窝,41,2020-12-31 07:58,做好个人防护，中国这种做法不可能大面积传染开,上海 宝山区,0.122701,0.877299
2,尼奇窝窝,11,2020-12-3 09:46,保护好自己，不给国家添乱,上海 宝山区,0.602974,0.397026
3,尼奇窝窝,9,2020-11-30 09:01,医护人员辛苦了,上海 宝山区,0.971032,0.028968
4,尼奇窝窝,3,2020-11-22 10:48,太不容易了,上海 宝山区,0.612186,0.387814
...,...,...,...,...,...,...,...
22471,薏苡啊,0,2020-3-8 17:01,我是黄冈的 能把快递通了吗？我姨妈巾都没得用了,其他,0.084605,0.915395
22472,温良初久,0,2020-3-8 17:00,只希望不要因为海外输入，病毒再次卷土重来，不知道山东靠海地区，能不能做...,山东,0.022441,0.977559
22473,小森宝的小佳宝DZ,0,2020-3-8 17:00,得 好不容易零增长了 现在又加上个本土 服了服了 那些看着中国疫...,山东 济南,0.000020,0.999980
22474,李土條,0,2020-3-8 16:59,太好了！,其他,0.944181,0.055819


In [29]:
data.to_csv(path_or_buf="基本数据.csv",index=False)

In [33]:
final = pd.read_csv("最终数据.csv")
final

,人名,点赞数,时间,评论,地区,积极数据,消极数据
0,是吴珉珉呀,123,2020/12/31,辽宁疫情快结束吧！2021好运来！,北京 海淀区,0.740446,0.259554
1,尼奇窝窝,41,2020/12/31,做好个人防护，中国这种做法不可能大面积传染开,上海 宝山区,0.122701,0.877299
2,尼奇窝窝,11,2020/12/3,保护好自己，不给国家添乱,上海 宝山区,0.602974,0.397026
3,尼奇窝窝,9,2020/11/30,医护人员辛苦了,上海 宝山区,0.971032,0.028968
4,尼奇窝窝,3,2020/11/22,太不容易了,上海 宝山区,0.612186,0.387814
...,...,...,...,...,...,...,...
22471,薏苡啊,0,2020/3/8,我是黄冈的???能把快递通了吗？我姨妈巾都没得用了,其他,0.084605,0.915395
22472,温良初久,0,2020/3/8,只希望不要因为海外输入，病毒再次卷土重来，不知道山东靠海地区，能不能做...,山东,0.022441,0.977559
22473,小森宝的小佳宝DZ,0,2020/3/8,得??好不容易零增长了??现在又加上个本土 服了服了??那些看着中国疫...,山东 济南,0.000020,0.999980
22474,李土條,0,2020/3/8,太好了！,其他,0.944181,0.055819
